<a href="https://colab.research.google.com/github/HmxaaK/Maven-Market-Report/blob/main/Macroeconomics_Indicator_Forecasting_Dashboard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Libraries
import pandas as pd
import numpy as np
from statsmodels.tsa.ar_model import AutoReg,ar_select_order
from statsmodels.tsa.ardl import ARDL,ardl_select_order

#Dataset
data = pd.read_csv('Macro Vars.csv', parse_dates=['Date'], index_col='Date')
data = data.asfreq('MS')

#Model Specified
y = np.log(data["PCE"]).diff().dropna()
X = np.log(data[['DSPI']]).diff().dropna()

#Optimal AR(p) selected
def AR_p_selection(target):
  max_p = int(np.sqrt(len(target)))
  lag_length = ar_select_order(target,maxlag=max_p,ic='aic')
  p = lag_length.ar_lags
  return p

#AR Model Fitted
p =  AR_p_selection(y)
ar_model = AutoReg(y,lags=p).fit()
print (f'Best AR(p) Model According to AIC Criteira : AR({p[-1]})')
forecast_AR = ar_model.predict(start=0,end=len(y))
residuals_AR = y - forecast_AR

#Optimal ARDL(p,q) selected
def ardl_pq_selection(target, features):
    max_p = int(np.sqrt(len(target)))
    max_q = int(np.sqrt(len(target)))
    lag_length = ardl_select_order(target, maxlag=max_p, exog=features,
                                maxorder=max_q, trend='c')
    p = lag_length.ar_lags
    order_dict = lag_length.dl_lags
    q = order_dict['DSPI']
    q = [var for var in q if var>0]
    return p,q

#ARDL Model Fitted
p,q = ardl_pq_selection(y,X)
ardl_model = ARDL(endog=y,lags=p,exog=X,order=q).fit()
print (f'Best ARDL(p,q) Model According to AIC Criteira : ARDL({p[-1]},{q[-1]})')
forecast_ARDL = ardl_model.predict(start=0,end=len(y))
residuals_ARDL = y - forecast_ARDL

#Combined into a Table
AR = pd.concat([forecast_AR,residuals_AR],axis=1)
AR['Model ID'] = 1
ARDL = pd.concat([forecast_ARDL,residuals_ARDL],axis=1)
ARDL['Model ID'] = 2
forecast_table = pd.concat([AR,ARDL])
forecast_table.columns = ['Forecast','Residuals','Model ID']
forecast_table.to_csv('Forecasts.csv')


Best AR(p) Model According to AIC Criteira : AR(10)
Best ARDL(p,q) Model According to AIC Criteira : ARDL(2,3)
